In [ ]:
import phd

import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nengo
import numpy as np
import soundfile as sf
from IPython.display import Audio

# Some plotting niceties
phd.plots.setup()

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
model = phd.sermo.Production()
model.trial.sequence = 'BLAA*POS1 + TII*POS2 + DAS*POS3'
freqs = [('ges-de-ccv', 'blaa', 1.8), ('ges-de-cv', 'tii', 2.2), ('ges-de-cvc', 'das', 1.5)]
t = 0.2
for gdir, ges, freq in freqs:
    path = phd.ges_path(gdir, '%s.ges' % ges.lower())
    traj = phd.vtl.parse_ges(path).trajectory(dt=model.trial.dt)
    model.add_syllable(label=ges.upper(), freq=freq, trajectory=traj)
    t += 1. / freq

In [ ]:
net = model.build(net=nengo.Network(seed=0))
with net:
    p_syll = nengo.Probe(net.sequence.syllable.output, synapse=0.01)
    p_gate = nengo.Probe(net.sequencer.gate, synapse=0.01)
    p_reset = nengo.Probe(net.sequencer.reset, synapse=0.01)
    p_timer = nengo.Probe(net.sequencer.timer, synapse=0.01)
    p_disinhib = [nengo.Probe(dmp.disinhibit, synapse=0.01) for dmp in net.syllables]
    p_dmps = [nengo.Probe(dmp.osc, synapse=0.01) for dmp in net.syllables]
    p_prod = nengo.Probe(net.production_info.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(t)

In [ ]:
syllables = [s[1].upper() for s in freqs]
plt.plot(sim.trange(), 
         nengo.spa.similarity(sim.data[p_syll],
                              net.sequence.vocab.create_subset(syllables)))
plt.legend(syllables, loc='best')

In [ ]:
plt.plot(sim.trange(), sim.data[p_gate])
plt.plot(sim.trange(), sim.data[p_reset])
plt.plot(sim.trange(), sim.data[p_timer])
plt.legend(['gate', 'kick', 'timer', 'timer'], loc="best")

In [ ]:
for p_dis in p_disinhib:
    plt.plot(sim.trange(), sim.data[p_dis])
plt.legend(syllables, loc="best")

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(sim.data[p_timer].T[0], sim.data[p_timer].T[1])
for i, p_dmp in enumerate(p_dmps):
    plt.subplot(2, 2, i + 2)
    plt.plot(sim.data[p_dmp].T[0], sim.data[p_dmp].T[1])

In [ ]:
plt.figure(figsize=(6, 4))
phd.plots.plot_traj(sim.data[p_prod], cbar=False);

## Running an experiment

In [ ]:
model = phd.sermo.Production()
model.syllable.tau = 0.018
expt = phd.experiments.ProductionExperiment(model, n_syllables=2, sequence_len=2)
key = expt.run()
res = phd.experiments.ProductionResult.load(key)

In [ ]:
# Actual trajectory (from .ges files)
img(res.traj)
print(res.seq)
print(res.freqs)

In [ ]:
# Simulated trajectory
img(res.simtraj)
print(res.simrmse)

In [ ]:
# Trajectory from .ges file reconstructed from simulated trajectory
img(res.reconstructed)
print(res.reconstructedrmse)

In [ ]:
Audio(data=res.clean_audio.flatten(), rate=res.fs)

In [ ]:
# Audio synthesized by VTL given .ges file
# reconstructed from simulated trajectory;
# note, no f0 gestures provided.
Audio(data=res.audio.flatten(), rate=res.fs)

In [ ]:
print("accuracy: %f" % res.accuracy)
print("timing: %f +/- %f" % (float(res.timing_mean), float(res.timing_var)))
print("cooccurring: %f, chance is %f" % (float(res.cooccur), float(res.co_chance)))

## Plotting experimental results

In [ ]:
# Put figures in phd/figures
root = os.path.abspath(os.path.join(os.path.dirname(phd.__file__), ".."))
def fig(name, ext='svg'):
    return os.path.join(root, "figures", "results", "%s.%s" % (name, ext))

def time_plots(key, x_label):
    df = phd.analysis.load_results(phd.experiments.ProductionResult, [key])

    cmp_args = {'group_by': key,
                'x_label': x_label}

    acc = plt.figure()
    phd.plots.timeseries(df,
                      columns=['accuracy'],
                      x_keys=['Accuracy'],
                      y_label='Accuracy',
                      **cmp_args)
    plt.axhline(phd.analysis.gs_accuracy_baseline, c='k', ls=':')
    plt.xlabel("")
    plt.tight_layout()
    acc.savefig(fig('prod-%s-acc' % key))

    err = plt.figure()
    phd.plots.timeseries(df,
                      columns=['n_sub', 'n_del', 'n_ins'],
                      x_keys=['Substitutions', 'Deletions', 'Insertions'],
                      y_label='Count',
                      **cmp_args)
    plt.xlabel("")
    plt.tight_layout()
    err.savefig(fig('prod-%s-err' % key))

    timing = plt.figure()
    phd.plots.timeseries(df,
                      columns=['timing_mean', 'timing_var'],
                      x_keys=['Timing mean', 'Timing variance'],
                      y_label='Timing',
                      **cmp_args)
    plt.xlabel("")
    plt.tight_layout()
    timing.savefig(fig('prod-%s-timing' % key))

    co = plt.figure()
    phd.plots.timeseries(df,
                      columns=['cooccur'],
                      x_keys=['Cooccurrence'],
                      y_label='Cooccurrence',
                      **cmp_args)
    plt.axhline(df["co_chance"].mean(), c='k', ls=":")
    plt.xlabel("")
    plt.tight_layout()
    co.savefig(fig('prod-%s-co' % key))
    return acc, timing, co

def cmp_plots(key, x_label, hue_order):
    df = phd.analysis.load_results(phd.experiments.ProductionResult, [key])

    cmp_args = {'group_by': key,
                'x_label': x_label,
                'hue_order': hue_order}

    acc = plt.figure()
    phd.plots.compare(df,
                      columns=['accuracy'],
                      x_keys=['Accuracy'],
                      y_label='Accuracy',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.axhline(phd.analysis.gs_accuracy_baseline, c='k', ls=':')
    plt.xlabel("")
    plt.tight_layout()
    acc.savefig(fig('prod-%s-acc' % key))

    err = plt.figure()
    phd.plots.compare(df,
                      columns=['n_sub', 'n_del', 'n_ins'],
                      x_keys=['Substitutions', 'Deletions', 'Insertions'],
                      y_label='Count',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.xlabel("")
    plt.tight_layout()
    err.savefig(fig('prod-%s-err' % key))

    timing = plt.figure()
    phd.plots.compare(df,
                      columns=['timing_mean', 'timing_var'],
                      x_keys=['Timing mean', 'Timing variance'],
                      y_label='Timing',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.xlabel("")
    plt.tight_layout()
    timing.savefig(fig('prod-%s-timing' % key))

    co = plt.figure()
    phd.plots.compare(df,
                      columns=['cooccur'],
                      x_keys=['Cooccurrence'],
                      y_label='Cooccurrence',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.axhline(df["co_chance"].mean(), c='k', ls=":")
    plt.xlabel("")
    plt.tight_layout()
    co.savefig(fig('prod-%s-co' % key))
    return acc, timing, co

In [ ]:
time_plots('tau', 'Syllable tau');

In [ ]:
time_plots('syllneurons', 'Syllable neurons');

In [ ]:
time_plots('seqneurons', 'Sequencer neurons');

In [ ]:
time_plots('freq', 'Frequency');

In [ ]:
time_plots('n_syllables', 'Syllabary size');

In [ ]:
time_plots('sequence_len', 'Sequence length');

In [ ]:
cmp_plots('repeat', 'Repeated syllables', hue_order=["False", "True"]);

In [ ]:
df = phd.analysis.load_results(
    phd.experiments.ProductionResult,
    keys=['tau'])

for i in range(len(df.index)):
    # Output all the .wavs to current directory
    cpath, spath = '%d-clean.wav' % i, '%d-sim.wav' % i
    if not os.path.exists(cpath):
        sf.write(df['clean_audio'][i], cpath, df['fs'][i])
    if not os.path.exists(spath):
        sf.write(df['audio'][i], spath, df['fs'][i])

# After going through them manually...
good = [0]
okay = [1]
bad = [2]
silent = [3]
total = len(good) + len(okay) + len(bad) + len(silent)
print("Good: %f" % (float(len(good)) / total))
print("Okay: %f" % (float(len(okay)) / total))
print("Bad: %f" % (float(len(bad)) / total))
print("Silent: %f" % (float(len(silent)) / total))

## Scaling

In [ ]:
def n_neurons(msg, model, n_syllables):
    net = model.build()
    nn = sum(e.n_neurons for e in net.all_ensembles)
    print("=== %s ===" % msg)

    # --- SPA sequence stuff
    spasyllneurons = model.sequence.syllable_d * model.sequence.n_per_d
    spaneurons = spasyllneurons * 5
    spaneurons += len(net.sequence.bind.all_ensembles) * model.sequence.n_per_d
    # * 5 = sequence, pos (2), pos_next (2), 
    # net.sequence.bind.all_ensembles => .5 * n_per_d per ensemble, but there's both bind and bind_next
    amneurons = 2 * ((n_syllables * 3 + 1) * 50)  # 50 is default for AM
    # 2: syllable and syllable_next
    # + 1 is for the default output
    spaneurons += amneurons

    # --- Sequencer stuff
    # * 2 + 2: timer is 2d; timer_recur is 2d
    # + 60: reset is 60 neurons
    # + 20: tr_inhibit is 20 neurons
    # + 60: gate is 60 neurons
    seqneurons = (model.sequencer.n_per_d * (2 + 2)) + 60 + 20 + 60

    # --- DMP stuff
    syllneurons = model.syllable.n_per_d
    # * (2 + 1): osc is 2d; diff is 1
    # + 20: diff_inhib is 20
    # + 20: inhib is 20
    dmpneurons = n_syllables * ((syllneurons * (2 + 1)) + 20 + 20)

    # Emulate having more than 3 syllables

    # --- Readout
    prodneurons = model.production_info.n_per_d * 48

    # --- Total
    allneurons = spaneurons + seqneurons + dmpneurons + prodneurons

    print("Nspa: %d, Nseq: %d, Ndmp: %d, Nprod: %d" % (
        spaneurons, seqneurons, dmpneurons, prodneurons))
    print("Total: %d neurons" % allneurons)
    print("")

def add_syllables(model, n_syllables):
    model.trial.sequence = "POS1"
    for p, f in zip(*phd.experiments.get_syllables(n_syllables, 1, 1)):
        tr = phd.vtl.parse_ges(p).trajectory(model.trial.dt)
        lbl = phd.experiments.path2label(p)
        model.add_syllable(label=lbl, freq=f, trajectory=tr)

model = phd.sermo.Production()
add_syllables(model, 3)
net = model.build()
n_neurons("Default configuration", model, 3)
model = phd.sermo.Production()
model.sequence.syllable_d = 256
add_syllables(model, 3)
net = model.build()
n_neurons("Conservative estimate", model, 2000)
model = phd.sermo.Production()
model.sequence.syllable_d = 512
add_syllables(model, 3)
net = model.build()
n_neurons("Generous estimate", model, 4000);